#  **_Applied Data Science Capstone_**
## Peer Graded Assignment:
## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada


This file is to present **_<span style="color: blue;">the part 1</span>_** of the peer graded assignment.

### <span style="color: blue;">Part 1: Transforming Wikipadia Table into Pandas Data Frame:</span>

> **_For this assignment, you will be required to explore and cluster the neighborhoods in Toronto._**

>   * Start by creating a new Notebook for this assignment.
>   * Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

> **_To create the dataframe:

>   * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
>   * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
>   * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
>   * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
>   * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
>   * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.



Download dependencies that I will use.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### <span style="color: blue;"> **_Scraping Wikipedia_** with Pandas </span>
    
Source: https://medium.com/swlh/how-to-scrape-a-website-with-a-single-line-of-python-code-5efe124275bb
    
> _"The Pandas library in Python includes a web scraper that pulls HTML table data into a dataframe in a single step. Simply insert the URL into the read_html() method and assign the resulting object to a variable so you can work with it. ... The catch is that this works only for text that has been put into table format in the underlying HTML. You can inspect the HTML to see if the data is in table form or you can just eyeball it. If it appears to be formatted in rows and columns, it’s likely to be table data. Wikipedia can be a great place to use the Pandas read_html()tool."_

In [2]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

Now, let's check the number of Table in the website.

In [3]:
print("# of Tables: ", len(data))

# of Tables:  3


Let's check 1) the shape and 2) the content of the first table.

In [4]:
print("Shape of the first Table: ", data[0].shape)
df=data[0]
df.head()

Shape of the first Table:  (180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Yes, the first table is what we want.
Let's remove all the rows with 'Not assigned' in the column 'Neighbourhood'.

 * Here, we use **_Boolean Indexing_** to select all those assigned 'Borough'.

In [5]:
df_selected=df[df['Borough'] != 'Not assigned']
df_selected.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


> **_More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma_** 


df2 = df1.groupby(['Postcode','Borough'], sort=False).agg(', '.join)


In [6]:
df_selected_Grouped = df_selected.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df_selected_Grouped.reset_index(inplace=True)
print('type', type(df_selected_Grouped))
df_selected_Grouped.head() 

type <class 'pandas.core.frame.DataFrame'>


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

Accordingly, I will use np.where(condition, A, B) to replace 'Not assigned' in 'Neighbout' with 'Brough' name.
 * if the condition is satisfied, A is executed;
 * if the condition is breached, B is executed.

Source: https://numpy.org/doc/stable/reference/generated/numpy.where.html


In [7]:
df_selected_Grouped["Neighbourhood"] = np.where(df_selected_Grouped["Neighbourhood"]== "Not Assigned", df_selected_Grouped["Borough"], df_selected_Grouped["Neighbourhood"])
df_selected_Grouped.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now, let's check the shape of the cleaned dataframe

In [8]:
df_selected_Grouped.shape

(103, 3)